In [1]:
import numpy as np
import pandas as pd

In [2]:
df_final_clean = pd.read_csv('df_final_clean.csv')
df_final_clean_sorted = df_final_clean.sort_values(by='timestamp')
df_final_clean_sorted[['timestamp', 'user_id', 'post_id', 'target']].head(3)

,timestamp,user_id,post_id,target
795415,2021-10-01 06:01:40,59784,307,1
988115,2021-10-01 06:01:40,73455,2671,1
295082,2021-10-01 06:01:52,21049,1453,0


In [3]:
df_final_clean_sorted['topic_hour'] = df_final_clean_sorted['topic_code'] * df_final_clean_sorted['hour_cos']
df_final_clean_sorted['age_topic'] = df_final_clean_sorted['age_category'] * df_final_clean_sorted['topic_code']
df_final_clean_sorted['weekend_hour'] = (df_final_clean_sorted['dow_sin'] > 0) * df_final_clean_sorted['hour_cos']

In [4]:
df_final_clean_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2316912 entries, 795415 to 706797
Data columns (total 23 columns):
 #   Column               Dtype  
---  ------               -----  
 0   timestamp            object 
 1   user_id              int64  
 2   post_id              int64  
 3   target               int64  
 4   age_category         int64  
 5   gender               int64  
 6   exp_group            int64  
 7   hour_sin             float64
 8   hour_cos             float64
 9   dow_sin              float64
 10  dow_cos              float64
 11  topic_code           int64  
 12  topic_movie          float64
 13  topic_politics       float64
 14  topic_sport          float64
 15  topic_business       float64
 16  topic_entertainment  float64
 17  topic_tech           float64
 18  post_embed_0         float64
 19  post_embed_1         float64
 20  topic_hour           float64
 21  age_topic            int64  
 22  weekend_hour         float64
dtypes: float64(14), int64(8), object(

In [5]:
print("\nЗначения и их частота:")
print(df_final_clean_sorted['age_topic'].value_counts().sort_index())


Значения и их частота:
age_topic
0     482519
1     109855
2     109096
3     283737
4     165053
5     111592
6     173936
8      83972
9     168076
10     75879
12    227878
15    134068
16     33670
18     35924
20     62188
24     20409
25     20761
30     14867
36      3432
Name: count, dtype: int64


In [6]:
def hitrate_at_5_fast(preds, train_data):
    """Более быстрая версия Hitrate@5."""
    y_true = train_data.get_label()
    groups = train_data.get_group()
    
    hitrates = []
    ptr = 0
    
    for group_size in groups:
        if group_size >= 5:  # Если в группе хотя бы 5 элементов
            # Берем индексы top-5 предсказаний в группе
            group_preds = preds[ptr:ptr + group_size]
            group_true = y_true[ptr:ptr + group_size]
            
            # Используем argsort вместо argpartition
            top_5_idx = np.argsort(-group_preds)[:5]
            hit = 1 if np.any(group_true[top_5_idx] == 1) else 0
            hitrates.append(hit)
        elif group_size > 0:
            # Для групп меньше 5 элементов считаем по всем
            group_true = y_true[ptr:ptr + group_size]
            hit = 1 if np.any(group_true == 1) else 0
            hitrates.append(hit)
        
        ptr += group_size
    
    return 'hitrate@5', np.mean(hitrates), True

In [7]:
import lightgbm as lgb

# ============================================================================
#  0. ЗАГРУЗКА И СПЛИТ
# ============================================================================

df = df_final_clean_sorted.copy()
df['timestamp'] = pd.to_datetime(df['timestamp'])
split_time = pd.Timestamp("2021-12-12 00:59:05.600000")

df_train = df[df['timestamp'] < split_time].copy()
df_test = df[df['timestamp'] >= split_time].copy()

# Фичи
exclude = ['timestamp', 'user_id', 'post_id', 'target']
feature_cols = [c for c in df_train.columns if c not in exclude]

# Автоматическое определение числовых и категориальных
numeric_cols = df_train.select_dtypes(include=[np.number]).columns
cat_cols = ['age_category', 'gender', 'topic_code', 'exp_group', 'age_topic',]
num_cols = [c for c in numeric_cols if c not in cat_cols and c not in exclude]

# Быстрая нормализация
if num_cols:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df_train[num_cols] = scaler.fit_transform(df_train[num_cols])
    df_test[num_cols] = scaler.transform(df_test[num_cols])
    print(f"Нормализовано {len(num_cols)} числовых фичей")


# ============================================================================
# 1. ПОДГОТОВКА ДАННЫХ ДЛЯ LAMBDARANK
# ============================================================================

print("="*60)
print("ПОДГОТОВКА ДАННЫХ ДЛЯ LAMBDARANK")
print("="*60)

# 1.1 Сортируем по user_id (ОБЯЗАТЕЛЬНО!)
df_train = df_train.sort_values('user_id').reset_index(drop=True)
df_test = df_test.sort_values('user_id').reset_index(drop=True)

print(f"После сортировки: train={len(df_train)} записей, {df_train['user_id'].nunique()} пользователей")
print(f"                 test={len(df_test)} записей, {df_test['user_id'].nunique()} пользователей")

# 1.2 Вычисляем размеры групп (количество постов на пользователя)
train_group_sizes = df_train.groupby('user_id').size().values
test_group_sizes = df_test.groupby('user_id').size().values

print(f"\nРазмеры групп train: {train_group_sizes[:5]}... (первые 5)")
print(f"Всего групп в train: {len(train_group_sizes)}")
print(f"Средний размер группы: {train_group_sizes.mean():.1f}")

# Проверка
assert len(train_group_sizes) == df_train['user_id'].nunique()
assert len(test_group_sizes) == df_test['user_id'].nunique()


# 1.4 Категориальные фичи
categorical_cols = ['age_category', 'gender', 'topic_code', 'exp_group', 'age_topic',]
for col in categorical_cols:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

# 1.5 Отделяем X, y
X_train = df_train[feature_cols]
X_test = df_test[feature_cols]
y_train = df_train['target'].values
y_test = df_test['target'].values

# ============================================================================
# 2. СОЗДАНИЕ DATASET С  ГРУППАМИ
# ============================================================================

print("\n" + "="*60)
print("СОЗДАНИЕ LIGHTGBM DATASET")
print("="*60)

# Критически важно: group=group_sizes, а не group=GroupBy объект!
train_data = lgb.Dataset(
    X_train,  # DataFrame с category типами
    label=y_train,
    group=train_group_sizes,  # МАССИВ размеров групп!
    categorical_feature=categorical_cols,
    free_raw_data=False
)

valid_data = lgb.Dataset(
    X_test,
    label=y_test,
    group=test_group_sizes,  # МАССИВ размеров групп!
    reference=train_data,  # Важно для консистентности категорий
    categorical_feature=categorical_cols,
    free_raw_data=False
)

print(f"Создан train Dataset: {len(X_train)} строк, {len(train_group_sizes)} групп")
print(f"Создан valid Dataset: {len(X_test)} строк, {len(test_group_sizes)} групп")

Нормализовано 14 числовых фичей
ПОДГОТОВКА ДАННЫХ ДЛЯ LAMBDARANK
После сортировки: train=1869653 записей, 163105 пользователей
                 test=447259 записей, 134462 пользователей

Размеры групп train: [ 8 19 19  4  5]... (первые 5)
Всего групп в train: 163105
Средний размер группы: 11.5

СОЗДАНИЕ LIGHTGBM DATASET
Создан train Dataset: 1869653 строк, 163105 групп
Создан valid Dataset: 447259 строк, 134462 групп


In [8]:

# ============================================================================
# 3. ПАРАМЕТРЫ LAMBDARANK
# ============================================================================

# Рассчитываем веса
positive_count = y_train.sum()
negative_count = len(y_train) - positive_count
scale_pos_weight = negative_count / positive_count  # ≈ 4.0

print(f"Соотношение negative:positive = {scale_pos_weight:.1f}:1")

params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5, 10],
    
    # БАЛАНСИРОВКА
    'scale_pos_weight': scale_pos_weight,  # Автоматическая балансировка
    
    # Параметры lambdarank
    'lambdarank_truncation_level': 10,
    'lambdarank_norm': True,
    'label_gain': [0, 1],
    
    # Основные параметры (усилил регуляризацию)
    'learning_rate': 0.05,
    'num_leaves': 31,
    'min_data_in_leaf': max(20, int(20 * scale_pos_weight / 10)),  # Увеличил
    'max_depth': 7,  
    
    # Регуляризация
    'lambda_l1': 0.15,
    'lambda_l2': 0.15,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    
    'verbosity': -1,
    'seed': 42,
}

# ============================================================================
# 4. ОБУЧЕНИЕ
# ============================================================================

print("\n" + "="*60)
print("НАЧИНАЕМ ОБУЧЕНИЕ LAMBDARANK")
print("="*60)

model = lgb.train(
    params,
    train_data,
    num_boost_round=500,
    valid_sets=[valid_data],
    valid_names=['valid'],
    feval=hitrate_at_5_fast,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

print("\n" + "="*60)
print("✅ LAMBDARANK ОБУЧЕН!")
print("="*60)

Соотношение negative:positive = 5.1:1

НАЧИНАЕМ ОБУЧЕНИЕ LAMBDARANK
Training until validation scores don't improve for 50 rounds
[50]	valid's ndcg@5: 0.868313	valid's ndcg@10: 0.878286	valid's hitrate@5: 0.49231
Early stopping, best iteration is:
[24]	valid's ndcg@5: 0.86827	valid's ndcg@10: 0.878207	valid's hitrate@5: 0.492437

✅ LAMBDARANK ОБУЧЕН!


In [9]:
# ============================================================================
# 5. СОХРАНЕНИЕ МОДЕЛИ
# ============================================================================
print("\n3. Сохранение модели...")

model.save_model('lgbm_lambdarank_model.txt')
print(f"   Модель сохранена: lgbm_lambdarank_model.txt")

# ============================================================================
# 6. АНАЛИЗ РЕЗУЛЬТАТОВ
# ============================================================================
print("\n4. Анализ результатов...")

# Важность фичей
importance_gain = model.feature_importance(importance_type='gain')
importance_split = model.feature_importance(importance_type='split')

# Топ-20 фичей
importance_data = []
for i, name in enumerate(feature_cols):
    importance_data.append({
        'feature': name,
        'importance_gain': importance_gain[i],
        'importance_split': importance_split[i]
    })

importance_df = pd.DataFrame(importance_data)
importance_df = importance_df.sort_values('importance_gain', ascending=False)

print("\nТОП ВАЖНЫХ ФИЧЕЙ (по gain):")
print("="*60)
print(importance_df.head(40).to_string(index=False))


3. Сохранение модели...
   Модель сохранена: lgbm_lambdarank_model.txt

4. Анализ результатов...

ТОП-10 ВАЖНЫХ ФИЧЕЙ (по gain):
            feature  importance_gain  importance_split
          age_topic     13174.302507                82
       age_category     10953.729088                95
         topic_code      9459.585524                80
         topic_hour      5206.247795                52
     topic_politics      2894.606594                58
           hour_cos      2850.715607                31
        topic_sport      2563.668386                62
       post_embed_1      2025.047792                44
        topic_movie      2000.863907                48
       post_embed_0      1453.826305                40
           hour_sin      1149.651989                25
       weekend_hour       963.130211                14
topic_entertainment       818.134007                26
     topic_business       673.745003                25
          exp_group       380.377602         

"""

#Загрузка в базу курса
import pandas as pd
from sqlalchemy import create_engine
import time

def safe_chunked_upload(df, table_name, chunk_size=50000):  # Уменьшил чанк
    
    total_rows = len(df)
    total_chunks = total_rows // chunk_size + 1
    print(f"Всего строк: {total_rows:,}, чанков: {total_chunks}")
    
    start_time = time.time()
    
    for i, start in enumerate(range(0, total_rows, chunk_size)):
        chunk_start_time = time.time()
        end = min(start + chunk_size, total_rows)
        chunk = df.iloc[start:end]
        
        success = False
        retries = 3  # 3 попытки
        
        for attempt in range(retries):
            try:
                # КАЖДЫЙ РАЗ новое соединение
                engine = create_engine(
                    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
                    "postgres.lab.karpov.courses:6432/startml"
                )
                
                with engine.begin() as conn:
                    if start == 0 and attempt == 0:  # Только для первой попытки
                        chunk.to_sql(
                            table_name, 
                            con=conn, 
                            if_exists='replace', 
                            index=False,
                            chunksize=10000  # Внутренний chunksize
                        )
                    else:
                        chunk.to_sql(
                            table_name, 
                            con=conn, 
                            if_exists='append', 
                            index=False,
                            chunksize=10000
                        )
                
                engine.dispose()
                success = True
                break
                
            except Exception as e:
                print(f"Попытка {attempt+1}/{retries} не удалась: {e}")
                time.sleep(10 * (attempt + 1))  # Увеличивающаяся пауза
                continue
        
        if not success:
            print(f"❌ Чанк {i+1} не удалось записать после {retries} попыток")
            return False
        
        chunk_time = time.time() - chunk_start_time
        
        print(f"✅ Чанк {i+1}/{total_chunks} записан ({end:,}/{total_rows:,} строк)")
        print(f"   ⏱️ Время чанка: {chunk_time:.1f} сек")
        print(f"   📊 Прогресс: {(end/total_rows*100):.1f}%")
        
        # Пауза между чанками
        if i < total_chunks - 1:
            time.sleep(2)
    
    total_time = time.time() - start_time
    print(f"\n✅ Все данные успешно записаны!")
    print(f"⏱️ Общее время: {total_time/60:.1f} минут")
    
    return True

# Использование
if safe_chunked_upload(df_final_clean_sorted, 'g_seregin_lbm4969_features', 70000):
    print("🎉 Все данные успешно записаны!")
else:
    print("💥 Запись прервана из-за ошибки")

"""